In [1]:
import pandas as pd
import numpy as np
from sklearn import neighbors
import matplotlib.pyplot as plt
from pandas.tseries.holiday import USFederalHolidayCalendar as calendarUSFH


# Cargar set de datos

In [2]:
trip = pd.read_csv('data/trip_train.csv')

In [3]:
trip_test = pd.read_csv('data/trip_test.csv')

In [4]:
weather = pd.read_csv('data/weather.csv')

In [5]:
station = pd.read_csv('data/station.csv')

# Modificando set de datos

In [6]:
trip_test.sample()

,id,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
28765,563717,12/2/2014 18:27,2nd at South Park,64,12/2/2014 18:31,San Francisco Caltrain 2 (330 Townsend),69,109,Subscriber,94107


In [ ]:
stationReduced = station.loc[:,['id','name','city']]

def choseZipCode(city):
    if city == 'San Francisco' : return 94107
    elif city == 'San Jose' : return 95113
    elif city == 'Redwood City' : return 94063
    elif city == 'Palo Alto' : return 94301
    elif city == 'Mountain View' : return 94041

stationReduced['zip_code'] = stationReduced.loc[:,'city'].apply(choseZipCode)

stationReduced = stationReduced.drop(labels=['id','city'],axis=1)
stationReduced.rename(columns={'name':'station'},inplace=True)

In [29]:
trip.rename(columns={'start_station_name':'station'},inplace=True)
trip_test.rename(columns={'start_station_name':'station'},inplace=True)

In [8]:
trip_test = trip_test.drop(labels=['zip_code','end_date','end_station_name','end_station_id','bike_id'],axis=1)

In [9]:
trip = trip.drop(labels=['zip_code','end_date','end_station_name','end_station_id','bike_id'],axis=1)

In [10]:
weather['precipitation_inches'] = weather['precipitation_inches'].apply(lambda x : 0.005 if (x == 'T') else float(x))

In [11]:

weather = weather[weather.precipitation_inches.isnull() == False]

weather = weather.fillna(0)

weather['events'] = weather['events'].apply(lambda x : 'Normal' if x == 0 else x)
weather['events'] = weather['events'].apply(lambda x : 'Rain' if x == 'rain' else x)
weather['date'] = weather['date'].apply(lambda date : pd.to_datetime(date).date())

In [12]:
def splitDateAndTime(dateAndTime):
    splitLine = dateAndTime.split(' ')
    return str(splitDate(splitLine[0]))  + splitTime(splitLine[1])

def splitDate(date):
    lineSplit = date.split('/')
    return str(lineSplit[2]) + str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1] if int(lineSplit[1]) >= 10 else '0' + str(lineSplit[1]) )

def splitTime(time):
    lineSplit = time.split(':')
    return str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1])


In [13]:
trip['time'] = trip['start_date'].apply(lambda x : int(splitDateAndTime(x)[8:12]) )
trip_test['time'] = trip_test['start_date'].apply(lambda x : int(splitDateAndTime(x)[8:12]) )

In [14]:
trip.loc[:,'start_date'] = trip['start_date'].apply(lambda x : pd.to_datetime(splitDateAndTime(x)))
trip_test.loc[:,'start_date'] = trip_test['start_date'].apply(lambda x : pd.to_datetime(splitDateAndTime(x)))

In [15]:
trip.loc[:,'day_of_week'] = trip['start_date'].apply(lambda x : x.dayofweek )
trip_test.loc[:,'day_of_week'] = trip_test['start_date'].apply(lambda x : x.dayofweek )

In [16]:
trip.loc[:,'month'] = trip['start_date'].apply(lambda x : x.month )
trip_test.loc[:,'month'] = trip_test['start_date'].apply(lambda x : x.month )

In [17]:
trip.loc[:,'year'] = trip['start_date'].apply(lambda x : x.year )
trip_test.loc[:,'year'] = trip_test['start_date'].apply(lambda x : x.year )

In [18]:
trip.loc[:,'dayofyear'] = trip['start_date'].apply(lambda x : x.dayofyear )
trip_test.loc[:,'dayofyear'] = trip_test['start_date'].apply(lambda x : x.dayofyear )

In [19]:
trip.loc[:,'start_date'] = trip['start_date'].apply(lambda x : pd.to_datetime(x).date())
trip_test.loc[:,'start_date'] = trip_test['start_date'].apply(lambda x : pd.to_datetime(x).date())

In [20]:
cal = calendarUSFH()
holidays = cal.holidays(return_name=True,start=pd.to_datetime('20130101'), end=pd.to_datetime('20151231'))
holiday_festive_day = pd.DataFrame(holidays,columns=['holiday']).reset_index()
holiday_festive_day.rename(columns={'index':'start_date'},inplace=True)

In [21]:
holiday_festive_day.loc[:,'start_date'] = holiday_festive_day['start_date'].apply(lambda x : pd.to_datetime(x).date())

In [22]:
trip = pd.merge(holiday_festive_day,trip,on=['start_date'],how='right')
trip_test = pd.merge(holiday_festive_day,trip_test,on=['start_date'],how='right')

In [23]:
trip.loc[:,'is_holiday'] = trip.holiday.isnull()
trip_test.loc[:,'is_holiday'] = trip_test.holiday.isnull()

In [24]:
trip.loc[:,'is_holiday'] = trip['is_holiday'].apply(lambda x : not x )
trip_test.loc[:,'is_holiday'] = trip_test['is_holiday'].apply(lambda x : not x )

In [25]:
trip.loc[:,'subscription_type'] = trip['subscription_type'].apply(lambda x : 0 if ('Subscriber' == x) else 1 )
trip_test.loc[:,'subscription_type'] = trip_test['subscription_type'].apply(lambda x : 0 if ('Subscriber' == x) else 1 )

In [31]:
trip.sample()

,start_date,holiday,id,duration,station,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday
222313,2014-03-28,NaN,230605,784,Davis at Jackson,42,0,1740,4,3,2014,87,False


In [30]:
trip_test.sample()

,start_date,holiday,id,station,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday
102810,2015-01-27,NaN,619901,Temporary Transbay Terminal (Howard at Beale),55,0,853,1,1,2015,27,False


In [32]:
tripWithZipCode = pd.merge(trip,stationReduced,on=['station'],how='right')
tripWithZipCode_test = pd.merge(trip_test,stationReduced,on=['station'],how='right')

In [33]:
tripWithZipCode.sample()

,start_date,holiday,id,duration,station,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday,zip_code
308063,2013-09-27,NaN,37577,2187,Embarcadero at Sansome,60,0,1616,4,9,2013,270,False,94107


In [34]:
tripWithZipCode_test.sample()

,start_date,holiday,id,station,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday,zip_code
5330,2015-01-30,NaN,625960,2nd at Folsom,62,0,1751,4,1,2015,30,False,94107


In [35]:
tripWithZipCode = tripWithZipCode.drop(labels=['holiday'],axis=1).dropna()
tripWithZipCode_test = tripWithZipCode_test.drop(labels=['holiday'],axis=1).dropna()

In [36]:
tripWithZipCode.rename(columns={'start_date':'date'},inplace=True)

In [37]:
tripWithZipCode_test.rename(columns={'start_date':'date'},inplace=True)

In [38]:
tripsWithWeather = pd.merge(tripWithZipCode,weather,on=['date','zip_code'],how='left')
tripsWithWeather_test = pd.merge(tripWithZipCode_test,weather,on=['date','zip_code'],how='left')

In [39]:
tripsWithWeather.rename(columns={'start_station_id':'station_id'},inplace=True)
tripsWithWeather_test.rename(columns={'start_station_id':'station_id'},inplace=True)

In [40]:
tripsWithWeather = tripsWithWeather.dropna()
tripsWithWeather_test = tripsWithWeather_test.dropna()

In [41]:
tripsWithWeather = tripsWithWeather[tripsWithWeather['duration'] < 5000]

In [42]:
tripsWithWeather.shape

(527095, 35)

In [43]:
tripsWithWeather_test.shape

(117979, 34)

In [47]:
test_id = tripsWithWeather_test['id']

test_target = tripsWithWeather['duration'][-50000:]

In [48]:
train_target = tripsWithWeather['duration']

In [49]:
tripsWithWeatherReduced = tripsWithWeather.drop(labels=['duration','date','station','events','id'],axis=1)

In [51]:
tripsWithWeatherReduced_test = tripsWithWeather_test.drop(labels=['date','station','events','id'],axis=1)

In [60]:
train = tripsWithWeatherReduced
test = tripsWithWeatherReduced_test

# Reduccion de dimensiones 

In [53]:
from sklearn.decomposition import IncrementalPCA

In [54]:
pca = IncrementalPCA(n_components=8)

In [55]:
tripTransform = pca.fit_transform(tripsWithWeatherReduced)
tripTransform_test = pca.fit_transform(tripsWithWeatherReduced_test)

In [56]:
tripTransform.shape

(527095, 8)

In [57]:
tripTransform_test.shape

(117979, 8)

In [58]:
tripTransformDF = pd.DataFrame(tripTransform)
tripTransformDF_test = pd.DataFrame(tripTransform_test)

In [59]:
tripTransformDF = tripTransformDF.abs()
tripTransformDF_test = tripTransformDF_test.abs()

In [ ]:
train = tripTransformDF

In [ ]:
test = tripTransformDF_test

In [ ]:
train.shape

In [ ]:
train_target.shape

# Machine Learning

In [63]:
from sklearn import ensemble

In [64]:
rfr = ensemble.RandomForestRegressor(n_jobs=-1,n_estimators=50,criterion='mse',oob_score=True)

In [65]:
rfr.fit(train,train_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=-1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [66]:
test_prediction = rfr.predict(test)

### Conversion para entrega

In [72]:
predictionDF = pd.DataFrame(test_prediction,columns={"prediction"})

In [73]:
predictionDF.loc[:,'id'] = test_id.values

In [88]:
predictionDF.to_csv(path_or_buf='prediction',sep=',',header=True,columns=['id','prediction'],index=False)

### CrossValidation
#### En caso de querer usar parte del set de entrenamiento como set de prueba 

rfr.score(test,test_target)

from sklearn.metrics import mean_squared_error

mean_squared_error(test_target,test_prediction)

En caso de ser necesario de Cross Validation

predictionDF.loc[:,'spectate'] = test_target.values

def predictError(predict,spectate):
    if (spectate > predict) : return  ((spectate-predict)/spectate)*100
    else  :  return ((predict-spectate)/predict )*100

predictionDF['errorValue'] = predictionDF.apply(axis=1,func=lambda x : predictError(x[0],x[1]))

predictionDF.sample(5)

print 'El error promedio que comete Random Forest Regresion al predecir es de', float(predictionDF['errorValue'].mean()),'%'

print "El algoritmo Random Forest Regresion predice un ",\
float(predictionDF[predictionDF['errorValue'] < 15].shape[0]) / float(predictionDF.shape[0]) *100 ,\
"% de los valores, con un error menor al 15%"


predictionDF.describe()